In [1]:
# nltk related imports
from nltk.stem.snowball import SnowballStemmer # used as stemmer
from nltk.corpus import stopwords # used to remove stopwords
from nltk.corpus import wordnet # used to check symantic of the terms
from nltk import download # used to download wordnet and stopwords

# other packages needed
import numpy as np
import pandas as pd
import os
import re

In [2]:
# constants used in the project

DATASET_PATH = "./CISI.ALL"
DOCS_FOLDER = "./docs"
DICT_EXCEL_PATH = "./dict.xlsx"

In [3]:
# initalize what's needed

download('wordnet')
download('stopwords')

stop_words = stopwords.words('english')
stemmer = SnowballStemmer(language='english')

dictionary = []

# create the docs folder if not exist
os.makedirs(DOCS_FOLDER, exist_ok=True)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# utility functions

def read_dataset(file_name: str) -> str:
  """
    Function used to read the dataset details
  """
  dataset: str = ""
  with open(file_name, 'r') as f:
    dataset = f.read()
  return dataset


def parse_documents(dataset: str) -> list[dict]:
  """
    Function to parse dataset format and extract documents with their components
    and returns a list of dictionaries, each containing doc_index and doc_content
  """
  parsed_docs: list[dict] = []

  # skip first empty index
  doc_parts = re.split(r'\.I\s*(\d+)', dataset)[1:]

  # step is 2 inorder to check index and content in pairs
  for i in range(0, len(doc_parts), 2):
    if i+1 < len(doc_parts):
      doc_index = doc_parts[i].strip()
      doc_content = doc_parts[i+1].strip()

      # attributes we want to extract
      title = ""
      author = ""
      abstract = ""

      title_match = re.search(r'\.T(.*?)(?=\.[A-Z]|\Z)', doc_content, re.DOTALL)
      if title_match:
        title = title_match.group(1).strip()

      author_match = re.search(r'\.A(.*?)(?=\.[A-Z]|\Z)', doc_content, re.DOTALL)
      if author_match:
        author = author_match.group(1).strip()

      abstract_match = re.search(r'\.W(.*?)(?=\.[A-Z]|\Z)', doc_content, re.DOTALL)
      if abstract_match:
        abstract = abstract_match.group(1).strip()

      formatted_content = f"{title}\n{author}\n{abstract}"
      parsed_docs.append({"idx": doc_index, "content": formatted_content})

  return parsed_docs


def is_meaningful(word: str) -> bool:
  """
    Function to check if a word is meaningful symantically
  """
  return len(wordnet.synsets(word)) > 0


def is_valid_word(word: str) -> bool:
  """
    Function to check if a word in document is valid or not to be tokenized
  """
  return True if word.isalnum() and word not in stop_words and is_meaningful(word) else False


def process_doc(doc: str) -> str:
  """
    Function to receive document and stem its words and check if they are meaningful
    and add its words to global dictionary
  """
  global dictionary
  tokenized_doc = []

  for idx, word in enumerate(doc.lower().split()):
    if not is_valid_word(word):
      continue

    stemmed_word = stemmer.stem(word)
    tokenized_doc.append(stemmed_word)

    if not stemmed_word in dictionary:
      dictionary.append(stemmed_word)

  return ' '.join(tokenized_doc)

def calc_tf(doc: str, idx: int) -> None:
  """
    Function to receive document string and document index and calculate
    the Term Frequency for each of the words in the doc
  """
  global dictionary
  global tf_matrix

  document = doc.split()
  for word in document:
    if word in dictionary:
      frequency = document.count(word)
      row = dictionary.index(word)
      col = idx
      tf_matrix[row, col] = frequency


def calc_df(doc: str) -> None:
  """
    Function to receive document string and calculate the Document Frequency
    for each word of the document
  """
  global dictionary
  global idf_matrix

  document = set(doc.split())
  for word in document:
    if word in dictionary:
      row = dictionary.index(word)
      idf_matrix[row, 0] += 1


def matrix_to_excel(matrix, filename: str, cols: list[str], indices: list) -> None:
  """
    Function to receive a matrix and create a pandas dataframe out of it
    and convert that dataframe to excel output
  """
  df = pd.DataFrame(matrix, columns=cols, index=indices)
  df.to_excel(filename)

In [5]:
# STEM WORDS AND SYMANTIC CHECK WORDS IN DOCS AND CREATE NEW DOC FILES

dataset = read_dataset(DATASET_PATH)

# parse documents
documents = parse_documents(dataset)
doc_lengths = len(documents)

# iterate over documents and store the content of each doc in a separate file
doc_paths = []
print("# Starting writing docs to separate files...")
for doc in documents:
  idx = doc["idx"]
  content = doc["content"]

  doc_path = os.path.join("./docs", f"doc-{idx}.txt")
  formatted_doc = process_doc(content)

  with open(doc_path, "w") as f:
    f.write(formatted_doc)
  doc_paths.append(doc_path)
  print(f"## Document {idx} done processing!")

# Starting writing docs to separate files...
## Document 1 done processing!
## Document 2 done processing!
## Document 3 done processing!
## Document 4 done processing!
## Document 5 done processing!
## Document 6 done processing!
## Document 7 done processing!
## Document 8 done processing!
## Document 9 done processing!
## Document 10 done processing!
## Document 11 done processing!
## Document 12 done processing!
## Document 13 done processing!
## Document 14 done processing!
## Document 15 done processing!
## Document 16 done processing!
## Document 17 done processing!
## Document 18 done processing!
## Document 19 done processing!
## Document 20 done processing!
## Document 21 done processing!
## Document 22 done processing!
## Document 23 done processing!
## Document 24 done processing!
## Document 25 done processing!
## Document 26 done processing!
## Document 27 done processing!
## Document 28 done processing!
## Document 29 done processing!
## Document 30 done processing!
## D

In [6]:
# CONVERT DICTIONARY TO EXCEL FILE

df = pd.DataFrame(dictionary, columns=["Words"])
df.to_excel(DICT_EXCEL_PATH)

In [7]:
# TF/IDF Flow

tf_matrix = np.zeros((len(dictionary), doc_lengths))
idf_matrix = np.zeros((len(dictionary), 1))
tf_idf_matrix = np.zeros((len(dictionary), doc_lengths))

print("# Calculation of TF/IDF for documents started!")
for idx, doc_path in enumerate(doc_paths):
  with open(doc_path, "r") as f:
    doc = f.read()
    print(f"## Calculating TF for Doc-{idx+1}")
    calc_tf(doc, idx)
    print(f"## Calculating IDF for Doc-{idx+1}")
    calc_df(doc)

print("# Filling TF Matrix started!")
rows, columns = np.shape(tf_matrix)
for row in range(rows):
  for col in range(columns):
    if tf_matrix[row, col] > 0:
      target = tf_matrix[row, col]
      tf_matrix[row, col] = 1 + np.log10(target)
    else:
      tf_matrix[row, col] = 0
print("# Filling TF Matrix ended!")

print("# Filling IDF Matrix started!")
rows, _ = np.shape(idf_matrix)
for row in range(rows):
  if idf_matrix[row, 0] > 0:
    target = idf_matrix[row, 0]
    idf_matrix[row, 0] = np.log10((doc_lengths+1) / (target+1))
  else:
    idf_matrix[row, 0] = 0
print("# Filling IDF Matrix ended!")

print("# Filling TF/IDF Matrix started!")
tf_idf_matrix = tf_matrix * idf_matrix
print("# Filling TF/IDF Matrix ended!")

print("# Saving Excel Files for each matrix!")
doc_names = [path.split("/")[-1] for path in doc_paths]

matrix_to_excel(tf_matrix, "tf-data.xlsx", doc_names, dictionary)
matrix_to_excel(idf_matrix, "idf-data.xlsx", ["IDF"], dictionary)
matrix_to_excel(tf_idf_matrix, "tf-idf-data.xlsx", doc_names, dictionary)

print("# Project DONE :)")

# Calculation of TF/IDF for documents started!
## Calculating TF for Doc-1
## Calculating IDF for Doc-1
## Calculating TF for Doc-2
## Calculating IDF for Doc-2
## Calculating TF for Doc-3
## Calculating IDF for Doc-3
## Calculating TF for Doc-4
## Calculating IDF for Doc-4
## Calculating TF for Doc-5
## Calculating IDF for Doc-5
## Calculating TF for Doc-6
## Calculating IDF for Doc-6
## Calculating TF for Doc-7
## Calculating IDF for Doc-7
## Calculating TF for Doc-8
## Calculating IDF for Doc-8
## Calculating TF for Doc-9
## Calculating IDF for Doc-9
## Calculating TF for Doc-10
## Calculating IDF for Doc-10
## Calculating TF for Doc-11
## Calculating IDF for Doc-11
## Calculating TF for Doc-12
## Calculating IDF for Doc-12
## Calculating TF for Doc-13
## Calculating IDF for Doc-13
## Calculating TF for Doc-14
## Calculating IDF for Doc-14
## Calculating TF for Doc-15
## Calculating IDF for Doc-15
## Calculating TF for Doc-16
## Calculating IDF for Doc-16
## Calculating TF for Doc-1